In [1]:
import face_recognition as fr # To recognize faces
import numpy as np # To handle all lists/arrays
import os # To handle all matters relating to folders, paths, image/file names, etc.
import tkinter as tk
import csv
import pandas as pd
from dollarpy import Recognizer, Template, Point
import mediapipe as mp # Import mediapipe
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from queue import Empty
import base64
import json
from collections import deque
from imutils.video import VideoStream
import numpy as np
import cv2
from keras.models import model_from_json
import imutils
import time
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
from deepface import DeepFace
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
from keras.models import model_from_json


In [13]:

'''
with open('Patient_data.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            c = ['name' , 'status' , 'id' , 'recommended_exercises']
            writer.writerow(c)
'''

'''
with open('Results_From_Dollarpy.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            c = ['Original_Exercise' , 'Predicted_Exercise' , 'Accuracy']
            writer.writerow(c)
'''
'''
with open('For_ANOVA.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            c = ['Student','bicep_curl','lateral_raise','push_ups','tricep_extension','Best_result_exercise']
            writer.writerow(c)
'''

In [2]:
#Reading Phase
def reading_coordinates(csv_file_name):
      traj_info = pd.read_csv(csv_file_name)
      map_traj = {}
      for index, row in traj_info.iterrows():
            label = str(row['label'])
            trajectories = eval(row['trajectory'])
            trajectories = np.array(trajectories)
            map_traj[label] = trajectories
      return map_traj

In [3]:
def dollar_Templates(map_traj):
  Templates_dollar_py = []
  for k in map_traj:
    curr_traj = map_traj[k]
    L = []
    for c in curr_traj:
        p = Point(c[0] , c[1])
        L.append(p)
    print(k)
    print(L)
    template_dpy = Template(k , L)
    Templates_dollar_py.append(template_dpy)
  return Templates_dollar_py

   

In [4]:
import pandas as pd
def get_row_csv(filename , person_name):
    data = pd.read_csv(filename)
    row = data[data['name'] == person_name]
    return row

b = get_row_csv('Patient_data.csv' , 'o1')
print(b['recommended_exercises'].iloc[0])

bicep_curl,lateral_raise


In [5]:
import cv2
import numpy as np

def find_expression(frame):
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # load json and create model
    json_file = open('model/emotion_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    emotion_model = model_from_json(loaded_model_json)

    # load weights into new model
    emotion_model.load_weights("model/emotion_model.h5")
    print("Loaded model from disk")
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    return frame
    

In [6]:
import cv2, dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import winsound

IMG_SIZE = (64,56)
B_SIZE = (34, 26)
margin = 95
class_labels = ['center','left', 'right'] 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

font_letter = cv2.FONT_HERSHEY_PLAIN
model = load_model('models/gazev3.1.h5')
model_b = load_model('models/blinkdetection.h5')


def detect_gaze(eye_img):
    pred_l = model.predict(eye_img)
    accuracy = int(np.array(pred_l).max() * 100)
    gaze = class_labels[np.argmax(pred_l)]
    return gaze


def detect_blink(eye_img):
    pred_B = model_b.predict(eye_img)
    status = pred_B[0][0]
    status = status*100
    status = round(status,3)
    return  status

   
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int64)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

def get_Eye_Gaze(img):
    # pattern = []
    # frames = 10
    # pattern_length = 0
    frames_to_blink = 6
    blinking_frames = 0
    output = np.zeros((900,820,3), dtype="uint8")
    img = cv2.flip(img,flipCode = 1)
    h,w = (112,128)	
    global gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        

        for n in range(36,42):
            x= shapes.part(n).x
            y = shapes.part(n).y
            next_point = n+1
            if n==41:
                next_point = 36 
            
            x2 = shapes.part(next_point).x
            y2 = shapes.part(next_point).y
            cv2.line(img,(x,y),(x2,y2),(0,69,255),2)

        for n in range(42,48):
            x= shapes.part(n).x
            y = shapes.part(n).y
            next_point = n+1
            if n==47:
                next_point = 42 
            
            x2 = shapes.part(next_point).x
            y2 = shapes.part(next_point).y
            cv2.line(img,(x,y),(x2,y2),(153,0,153),2)
        shapes = face_utils.shape_to_np(shapes)
        #~~~~~~~~~~~~~~~~~56,64 EYE IMAGE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])
        #~~~~~~~~~~~~~~~~FOR THE EYE FINAL_WINDOW~~~~~~~~~~~~~~~~~~~~~~#
        eye_img_l_view = cv2.resize(eye_img_l, dsize=(128,112))
        eye_img_l_view = cv2.cvtColor(eye_img_l_view,cv2.COLOR_BGR2RGB)
        eye_img_r_view = cv2.resize(eye_img_r, dsize=(128,112))
        eye_img_r_view = cv2.cvtColor(eye_img_r_view, cv2.COLOR_BGR2RGB)
        #~~~~~~~~~~~~~~~~~FOR THE BLINK DETECTION~~~~~~~~~~~~~~~~~~~~~~~
        eye_blink_left = cv2.resize(eye_img_l.copy(), B_SIZE)
        eye_blink_right = cv2.resize(eye_img_r.copy(), B_SIZE)
        eye_blink_left_i = eye_blink_left.reshape((1, B_SIZE[1], B_SIZE[0], 1)).astype(np.float32) / 255.
        eye_blink_right_i = eye_blink_right.reshape((1, B_SIZE[1], B_SIZE[0], 1)).astype(np.float32) / 255.
        #~~~~~~~~~~~~~~~~FOR THE GAZE DETECTIOM~~~~~~~~~~~~~~~~~~~~~~~~#
        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_input_g = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        #~~~~~~~~~~~~~~~~~~PREDICTION PROCESS~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        status_l = detect_blink(eye_blink_left_i)
        gaze =  detect_gaze(eye_input_g)
        return gaze


In [7]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

from object_detection import ObjectDetection

In [2]:
op=ObjectDetection()

phonexy=[]
other=[]

cap = cv2.VideoCapture(0)

## Setup mediapipe instance
while cap.isOpened():
    ret, frame = cap.read()

    (class_id,scores,boxes)=op.detect(frame)
    
    #print(boxes)
    i=0
    for f in class_id:
        if f==67:
            b=boxes[i]
            (x,y,w,h)=b
            center = (int((x+w) / 2), int((y+h)/ 2))
            cv2.rectangle(frame ,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            phonexy.append(int(x))
            phonexy.append(int(y))
            break
        else:
            b=boxes[i]
            (x,y,w,h)=b
            cv2.rectangle(frame ,(x,y),(x+w,y+h),(255,0,0),2)
            other.append(int(x))
            other.append(int(y))
        i+=1
        
    cv2.imshow('Object', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading Object Detection
Running opencv dnn with YOLOv4


In [4]:
from dollarpy import Recognizer, Template, Point

i=0
temp=[]
while i<len(phonexy):
    temp.append(Point(phonexy[i],phonexy[i+1]))
    i+=2

i=0
temp2=[]
while i<len(other):
    temp2.append(Point(other[i],other[i+1]))
    i+=2

tmpl_1 = Template('Phone', temp)
#tmpl_2 = Template('Other', temp2)

global recognizer
recognizer = Recognizer([tmpl_1])
 

In [5]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

from object_detection import ObjectDetection

In [19]:
import time
def remove_identical_points(points):
    """
    This function eliminates consecutive identical points in a list.

    Args:
        points: A list of Points.

    Returns:
        A list of Points with consecutive identical points removed.
    """
    # Initialize the new points list with the first point
    new_points = [points[0]]
    
    # Iterate over the points, starting from the second point
    for i in range(1, len(points)):
        # If the current point is different from the last point in new_points, append it
        if points[i].x != new_points[-1].x or points[i].y != new_points[-1].y:
            new_points.append(points[i])
            
    return new_points


def tracking_classification(cap , rec , rec_prev ,  recommended , curr_exercise):
    op=ObjectDetection()
    reco=[]
    while cap.isOpened():
        ret, frame = cap.read()

        (class_id,scores,boxes)=op.detect(frame)
        i=0
        for f in class_id:
            if f==67:
                b=boxes[i]
                (x,y,w,h)=b
                p=Point(int(x),int(y))
                reco.append(p)
                break
            i+=1
        if reco:
            reco = remove_identical_points(reco)
            res = recognizer.recognize(reco)
            print('the object tracking result ' , res)
        cv2.imshow('Object', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    cap = cv2.VideoCapture(0)
    Test_Hand_Gesture(cap , rec_prev , recommended , curr_exercise , rec)

def first_face_recognition(video):
    # Path to folder containing all known faces. The 'known' folder needs all known faces, 
    # where the image\file name is the name of the person to be recognized.
    faces_path = "images"

    # Function to get face names, as well as face encodings
    def get_face_encodings():
        face_names = os.listdir(faces_path)
        face_encodings = []

        # For loop to retrieve all face encodings and store them in a list.
        # Below loop also gets the names of people and removes ".jpg", and stores
        # the names in a list
        for i, name in enumerate(face_names):
            face = fr.load_image_file(f"{faces_path}\\{name}")
            face_encodings.append(fr.face_encodings(face)[0])

            face_names[i] = name.split(".")[0] # To remove ".jpg" or any other image extension
        
        return face_encodings, face_names

    # Retrieving face encodings and storing them in the face_encodings variable, along with the names
    face_encodings, face_names = get_face_encodings()

    # Reference to webcam

    # Setting variable which will be used to scale size of image
    scl = 2

    ct = 1
    # Continuously capturing webcam footage
    while True:
        ct += 1
        success, image = video.read()

        # Making current frame smaller so program runs faster
        resized_image = cv2.resize(image, (int(image.shape[1]/scl), int(image.shape[0]/scl)))

        # Converting current frame to RGB, since that's what the face recognition module uses
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        # Retrieving face location coordinates and unknown encodings
        face_locations = fr.face_locations(rgb_image)
        unknown_encodings = fr.face_encodings(rgb_image, face_locations)
        # Iterating through each encoding, as well as the face's location
        for face_encoding, face_location in zip(unknown_encodings, face_locations):
            # Comparing known faces with unknown faces
            result = fr.compare_faces(face_encodings, face_encoding, 0.4)

            # Getting correct name if a match was found
            if True in result:
                name = face_names[result.index(True)]

                # Setting coordinates for face location
                top, right, bottom, left = face_location

                # Drawing rectangle around face
                cv2.rectangle(image, (left*scl, top*scl), (right*scl, bottom*scl), (0, 0, 255), 2)

                # Setting font, as well as displaying text of name
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(image, name, (left*scl, bottom*scl + 20), font, 0.8, (255, 255, 255), 1)
                video.release()
                cv2.destroyAllWindows()
                return name
                

        # Displaying final image on the screen
        cv2.imshow("frame", image)
        if(cv2.waitKey(1) & 0xFF == ord('q')):
            break
    video.release()
    cv2.destroyAllWindows()






# import the necessary packages
# define the lower and upper boundaries of the "green"
# ball in the HSV color space, then initialize the
# list of tracked points
'''
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
'''


def Test_Hand_Gesture(cap , recognizer , recommended , curr_exercise , rec_exercises):
    redLower = (169 , 148 , 92)
    redUpper = (179 , 255 ,255)

    greenLower = (19,123,102)
    greenUpper = (76 , 255 ,255)

    greenLower_tennis = (23,0,0)
    greenUpper_tennis = (79 , 231, 161)
    pts = deque(maxlen=64)  # Changed from args["buffer"]
    min_radius = 30  # minimum radius to consider an object
    min_area = 1000 # minimum area to consider an object
    trajectory_points = []
    trajectory_points_dp = []
    predicted_gesture = ''
    count = 0
    cr_ex_index = curr_exercise
    vid_name = recommended[cr_ex_index] + '.mp4'
    cap2 = cv2.VideoCapture(vid_name)
    starting_exercise = False
    starting_object_exercise = False
    # initialize video stream, allow camera to warm up
    vs = cv2.VideoCapture(0)
    ct = 0
    # keep looping
    while True:
        ct += 1
        # grab the current frame
        ret , frame = vs.read()
        if(ct % 60 == 0):
            if trajectory_points_dp and len(trajectory_points_dp) > 3:
                #trajectory_points_dp = remove_close_points(trajectory_points_dp , 1)
                trajectory_points_dp = remove_identical_points(trajectory_points_dp)
                result = recognizer.recognize(trajectory_points_dp)
                trajectory_points_dp = []
                predicted_gesture = result[0]
                if(predicted_gesture is not None):   
                    #print(predicted_gesture)      
                    if 'right' in predicted_gesture:
                        cr_ex_index += 1
                        if(cr_ex_index > len(recommended) - 1):
                            cr_ex_index = len(recommended) - 1
                        cr_ex = recommended[cr_ex_index] + '.mp4'
                        cap2.release()
                        cap2 = cv2.VideoCapture(cr_ex)
                        predicted_gesture = ''
                    if 'left' in predicted_gesture:
                        cr_ex_index -= 1
                        if(cr_ex_index < 0):
                            cr_ex_index = 0
                        cap2.release()
                        cr_ex = recommended[cr_ex_index] + '.mp4'
                        cap2 = cv2.VideoCapture(cr_ex)
                        predicted_gesture = ''
        # resize the frame, blur it, and convert it to the HSV
        # color space
        if ret:
            frame = imutils.resize(frame, width=600)
            frame = cv2.flip(frame, 1)
            if(ct % 40 == 0):
                frame = find_expression(frame)
                status = get_Eye_Gaze(frame)
                font = cv2.FONT_HERSHEY_SIMPLEX
                org_temp = (40, 40)
                fontScale = 2
                color_temp = (255,64,64)
                thickness = 1
                if(status is not None):
                    cv2.putText(frame, 'eye gaze:' + status, org_temp, font, 
                            fontScale, color_temp, thickness, cv2.LINE_AA) 
            blurred = cv2.GaussianBlur(frame, (11, 11), 0)
            hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
            
            
            # construct a mask for the color "green", then perform
            # a series of dilations and erosions to remove any small
            # blobs left in the mask
            mask = cv2.inRange(hsv, greenLower_tennis, greenUpper_tennis)
            mask = cv2.erode(mask, None, iterations=2)
            mask = cv2.dilate(mask, None, iterations=2)
            
            # find contours in the mask and initialize the current
            # (x, y) center of the ball
            cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            center = None
            
            # only proceed if at least one contour was found
            if len(cnts) > 0:
                # find the largest contour in the mask, then use
                # it to compute the minimum enclosing circle and
                # centroid
                c = max(cnts, key=cv2.contourArea)
                #((x, y), radius) = cv2.boundingRect(c)
                x, y, width, height = cv2.boundingRect(c)
                center = (int(x + width / 2), int(y + height / 2))
                radius = max(width, height) / 2
                #M = cv2.moments(c)
                #center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                
                # only proceed if the radius and area meet a minimum size
                if cv2.contourArea(c) > min_area:
                    # draw the circle and centroid on the frame,
                    # then update the list of tracked points
                    cv2.circle(frame, center, 5, (0, 0, 255), -1)
                    pts.appendleft(center)
                    trajectory_points.append([center[0],center[1]])
                    trajectory_points_dp.append(Point(center[0] , center[1]))

                    
            # update the points queue
            #pts.appendleft(center)
            
            # loop over the set of tracked points
            for i in range(1, len(pts)):
                # if either of the tracked points are None, ignore
                # them
                if pts[i - 1] is None or pts[i] is None:
                    continue
                
                # otherwise, compute the thickness of the line and
                # draw the connecting lines
                thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)
                cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
                
            # show the frame to our screen
            cv2.imshow("Frame", frame)
            ret2, frame2 = cap2.read()
            if not ret2:
                        # Reset the capture if the video has ended
                        cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)
            else:
                cv2.imshow('Second Video', frame2)
            key = cv2.waitKey(27) & 0xFF
            
            if(key == ord('s')):
                starting_exercise = True
                break
            if(key == ord('k')):
                 starting_object_exercise = True
                 break
                 
            # if the 'q' key is pressed, stop the loop
            if key == ord('q'):
             break
            '''
            if 'Start' in predicted_gesture:
                cap = cv2.VideoCapture(0)
                Test_dollar_py(cap , recognizer , recommended , cr_ex_index)
            '''
            


    # stop the camera video stream
    vs.release()

    # close all windows
    cv2.destroyAllWindows()
    if(starting_exercise == True):
        cap_temp = cv2.VideoCapture(0)
        Test_dollar_py(cap_temp , rec_exercises , recognizer , recommended , cr_ex_index)
    if(starting_object_exercise == True):
         cap_temp_2 = cv2.VideoCapture(0)
         tracking_classification(cap_temp_2 , rec_exercises , recognizer , recommended , cr_ex_index)
         





def Test_dollar_py(cap , rec , rec_prev ,  recommended , curr_exercise):
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)
    out = cv2.VideoWriter('Experiment.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, size)
    L2_point = []
    L_selected_points = [11,12,13,14,15,16]
    L_test_point = []
    m = {}
    amount_of_time = 180
    ground_truth = recommended[curr_exercise]
    map_exercises = {}
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:    
        ct = 0

        while(True):
                ct += 1
                ret, frame = cap.read()
                key = cv2.waitKey(27) & 0xFF
                if ret:
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image.flags.writeable = False        
                    results = holistic.process(image)
                    image.flags.writeable = True   
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    height, width, _ = frame.shape
                    for f in L_selected_points:
                        if(results.pose_landmarks is not None):         
                                        
                            x_cor = int(results.pose_landmarks.landmark[f].x * width)
                            y_cor = int(results.pose_landmarks.landmark[f].y * height)
                            
                            '''
                            x_cor = results.pose_landmarks.landmark[f].x
                            y_cor = results.pose_landmarks.landmark[f].y 
                            '''
                            p = Point(x_cor, y_cor)
                            L_test_point.append(p)  
                    
                    if ct % 3 == 0 and L_test_point:
                        print('helloooo there ' , L_test_point)
                        sub = L_test_point.copy()
                        sub = remove_identical_points(sub)
                        result = rec.recognize(sub)
                        res_str = result[0]
                        perc_str = str(round(result[1],3))
                        final_res = ""
                        if(res_str not in map_exercises):
                            map_exercises[res_str] = 1
                        else:
                            map_exercises[res_str] += 1
                        with open('Results_From_Dollarpy.csv', 'a+' , newline = '') as f:
                            writer = csv.writer(f)
                            row = [ground_truth , res_str , perc_str]
                            writer.writerow(row)
                        
                        checker = 'false'
                        if(res_str is not None):
                            if(res_str == ground_truth):
                                checker = 'true'
                            res_str = 'the label is ' + res_str
                        if(res_str is None):
                            res_str = 'the label is ' + 'None'
                            perc_str =  '0'
                        

                        font = cv2.FONT_HERSHEY_SIMPLEX
                        org = (50, 50)
                        fontScale = 1
                        color = (255, 0, 0)
                        thickness = 1
                        cv2.putText(frame, res_str, org, font, 
                                fontScale, color, thickness, cv2.LINE_AA)  
                        org2 = (50 , 100)
                        org3 = (50 , 150)
                        cv2.putText(frame, perc_str, org2, font, 
                                fontScale, color, thickness, cv2.LINE_AA)  
                        cv2.putText(frame, checker, org3, font, 
                                fontScale, color, thickness, cv2.LINE_AA)  
                        
                    if(ct % 50 == 0):
                        L_test_point = []
                        
                    out.write(frame)
                    cv2.imshow('Show',frame)    
                else:
                    break   
                if key == ord('q'):
                    break
                if key == ord('k'):
                    cap.release()
                    cv2.destroyAllWindows()
                    cap = cv2.VideoCapture(0)
                    Test_Hand_Gesture(cap, rec , recommended , c)


    cap.release()
    cv2.destroyAllWindows()
    cap = cv2.VideoCapture(0)
    print('The map of exercises ' , map_exercises)
    max_res = -99999999
    name_of_exercise = ''
    for d in map_exercises:
        if(map_exercises[d] > max_res):
            max_res = map_exercises[d]
            name_of_exercise = d
    
    best_res_string = str(max_res)
    with open('For_ANOVA.csv', 'a+' , newline = '') as f:
                            writer = csv.writer(f)
                            #row = ['S4' , str(map_exercises['bicep_curl']) , str(map_exercises['lateral_raise']), str(map_exercises['push_ups']), str(map_exercises['tricep_extension']), best_res_string]
                            #writer.writerow(row)
    
        
    Test_Hand_Gesture(cap , rec_prev , recommended , curr_exercise , rec)

 


curr_exercise = 0

map_traj = reading_coordinates('Gestures.csv')
temps = dollar_Templates(map_traj)
recognizer = Recognizer(temps)

map_traj_2 = reading_coordinates('Exercises_Data_whole.csv')
temps_2 = dollar_Templates(map_traj_2)
recognizer_2 = Recognizer(temps_2)

cap = cv2.VideoCapture(0)
name = first_face_recognition(cap)
row = get_row_csv('Patient_data.csv',name)
rec_exercseees = row['recommended_exercises'].iloc[0]
print(rec_exercseees)
recommended_exercises = rec_exercseees.split(',')
cap = cv2.VideoCapture(0)
#recommended_exercises = ['bicep_curl','lateral_raise','push_ups','tricep_extension']
Test_Hand_Gesture(cap , recognizer , recommended_exercises , curr_exercise , recognizer_2)

right_1
[(212, 163), stroke None, (214, 183), stroke None, (214, 172), stroke None, (214, 172), stroke None, (214, 172), stroke None, (214, 172), stroke None, (214, 165), stroke None, (214, 165), stroke None, (214, 165), stroke None, (214, 165), stroke None, (215, 167), stroke None, (215, 167), stroke None, (215, 167), stroke None, (215, 167), stroke None, (215, 165), stroke None, (215, 165), stroke None, (215, 165), stroke None, (215, 165), stroke None, (220, 171), stroke None, (220, 171), stroke None, (220, 171), stroke None, (220, 171), stroke None, (233, 179), stroke None, (233, 179), stroke None, (233, 179), stroke None, (233, 179), stroke None, (257, 188), stroke None, (257, 188), stroke None, (257, 188), stroke None, (257, 188), stroke None, (282, 197), stroke None, (282, 197), stroke None, (282, 197), stroke None, (282, 197), stroke None, (303, 186), stroke None, (303, 186), stroke None, (303, 186), stroke None, (303, 186), stroke None, (313, 162), stroke None, (313, 162), stro